In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
df_ports = pd.read_csv('data/processed/filtered_ports_with_temp.csv')
df_ports['MSL'] = np.nan
df_ports['SL_Trend'] = np.nan
df_ports.head()

,Year,Port,Yield,State,Region,Port_ID,Temp,MSL,SL_Trend
0,1981,"Brownsville-Port Isabel, TX",28.0,TX,0,0,21.083722,NaN,NaN
1,1982,"Brownsville-Port Isabel, TX",19.0,TX,0,0,21.838193,NaN,NaN
2,1983,"Brownsville-Port Isabel, TX",21.0,TX,0,0,22.162636,NaN,NaN
3,1984,"Brownsville-Port Isabel, TX",23.0,TX,0,0,21.856949,NaN,NaN
4,1985,"Brownsville-Port Isabel, TX",22.9,TX,0,0,21.438654,NaN,NaN


In [3]:
bad_ct = 0
not_one_ct = 0
total_ct = 0
cities_msl = {}
cities_tline = {}
for filename in os.listdir('data/slr'):
    df_raw = pd.read_csv('data/slr/{}'.format(filename), index_col=False)
    df_raw.columns = ['Year', 'Month', 'Monthly_MSL', 'Unverified', 'Linear_Trend', 'High_Conf.', 'Low_Conf.']
    df_raw = df_raw[df_raw['Year'] >= 1980]
    msl = {}
    tline = {}
    year_ct = {}
    city_msl = {}
    city_tline = {}
    for row in df_raw.itertuples():
        if(np.isnan(row.Monthly_MSL) or np.isnan(row.Linear_Trend)):
            bad_ct += 1
            total_ct += 1
            continue
        if(row.Month != 1):
            continue
        if(str(row.Year) not in msl):
            msl[str(row.Year)] = row.Monthly_MSL
            tline[str(row.Year)] = row.Linear_Trend
            year_ct[str(row.Year)] = 1.0
        else:
            msl[str(row.Year)] += row.Monthly_MSL
            tline[str(row.Year)] += row.Linear_Trend
            year_ct[str(row.Year)] += 1.0
        total_ct += 1
    for key in msl.keys():
        if(year_ct[key] != 1):
        city_msl[key] = msl[key] / year_ct[key]
        city_tline[key] = tline[key] / year_ct[key]
    cities_msl[filename[:filename.index('.csv')]] = city_msl
    cities_tline[filename[:filename.index('.csv')]] = city_tline

crap
crap
crap
crap
crap
crap
crap
crap
crap
crap
crap
crap
crap
crap
crap
crap
crap
crap
crap
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
510 1853


In [4]:
for row in df_ports.itertuples():
    if(row.Port in cities_msl.keys() and str(row.Year) in cities_msl[row.Port].keys()):
        df_ports.at[row.Index, 'MSL'] = cities_msl[row.Port][str(row.Year)]
        df_ports.at[row.Index, 'SL_Trend'] = cities_tline[row.Port][str(row.Year)]

In [5]:
df_ports = df_ports.drop(columns=['SL_Trend'])
df_ports.to_csv('data/processed/filtered_ports_w_temp_msl.csv', index=False)